In [1]:
%pwd

'/home/siddhu/Desktop/Text-Summarizer-With-HF/research'

In [2]:
import os

os.chdir("../")
%pwd

'/home/siddhu/Desktop/Text-Summarizer-With-HF'

In [ ]:
import zipfile
import urllib.request as request
from dataclasses import dataclass
from src.textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.logging import logger

In [4]:
@dataclass
class DataIngestionConfig:
    root_dir: str
    source_url: str
    local_data_file: str
    unzip_dir: str

In [5]:
class ConfigurationManager:
    
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,  
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename=self.config.local_data_file
            )
            logger.info("File downloaded successfully!")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}. Skipping download.")
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted successfully at {unzip_path}")

In [7]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)
data_ingestion.download_data()
data_ingestion.extract_zip_file()

[2026-01-31 09:12:41,454: INFO: common: YAML file 'config/config.yaml' read successfully.]
[2026-01-31 09:12:41,487: INFO: common: YAML file 'params.yaml' read successfully.]
[2026-01-31 09:12:41,488: INFO: common: Directory 'artifacts' created successfully or already exists.]
[2026-01-31 09:12:41,489: INFO: common: Directory 'artifacts/data_ingestion' created successfully or already exists.]
[2026-01-31 09:12:43,498: INFO: 2552295685: File downloaded successfully!]
[2026-01-31 09:12:43,554: INFO: 2552295685: File extracted successfully at artifacts/data_ingestion]
